In [1]:
import os
import pandas as pd
import numpy as np


In [2]:
#get list of files to rescrape
files = os.listdir(os.getcwd()+'/s2scrapedata')
files = [i for i in files if '__.csv' in i]

megadf = pd.DataFrame()

for f in files:
    tempdf = pd.read_csv('./s2scrapedata/'+f)
    megadf  = pd.concat((megadf,tempdf))

officialdf = pd.read_csv('../datadotgov_main.csv')
officialdf['ABN'] = officialdf['ABN'].astype(str)




In [3]:
#begin cleaning

megadf.index = megadf['ABN']
megadf = megadf.drop(['Website', 'Revenue', 'Expenses', 'Cause/s', 'Last_report_date', 'ref_religion','Unnamed: 0','ABN'], axis=1)



In [4]:
#add in interesting official data (namely no responsible persons, reg date and establishment date)

megadf = pd.merge(megadf,officialdf[["ABN",'Number_of_Responsible_Persons','Registration_Date','Date_Organisation_Established']],on='ABN',how='left')
megadf.to_csv('combined_scraped_list.csv')

In [5]:
#develop search/filter lists

# for free websites
freedomainlist = [  'webnode',
                    'wixsite',
                    'weebly',
                    'wordpress',
                    'hubspot',
                    'hs-sites',
                    'godaddy',
                    'site123',
                    'mozello',
                    'webstarts',
                    'jimdofree',
                    'ucraft',
                    'webflow',
                    'strinkingly',
                    'blog']
regexsearchstring = ''
for d in freedomainlist:
    regexsearchstring = (regexsearchstring+d+"|")[:-1]

webfilter_freesites     = megadf['website'].str.contains(pat=regexsearchstring,regex=True,na=False)
webfilter_free_orNone   = megadf['website'].str.contains(pat=regexsearchstring,regex=True,na=True)

# for religious reference
relfilter = megadf['religious_ref']==0

# for upto date reporting
recentreport_filter = megadf['lastreport'].str.contains('2021|2022',na=False)

# for state
statefilter = megadf['State']=='QLD'

# greater than one responsible person filter
teamsizefilter = megadf['Number_of_Responsible_Persons']>1

# name doesn't contain "trust" or "fund"
trustfundfilter  = ~megadf['Legal_Name'].str.contains('trust|fund|Trust|Fund|TRUST|FUND',regex=True,na=False)

# name doesn't contain "RSL", "Meals on Wheels" or'lifesave'
bigbusinessfilter  = ~megadf['Legal_Name'].str.contains('RSL|Returned And Services League|Returned Service League|Returned Services League|Rsl|Returned & Services League|Meals on Wheels|Meals On Wheels|Lifesave|Surf Life|Parents And Friends',regex=True,na=False)

In [6]:


filters = webfilter_free_orNone*relfilter*recentreport_filter*teamsizefilter*statefilter*trustfundfilter*bigbusinessfilter


megadf[filters].sort_values(by=['revenue'], ascending=False).iloc[91:110]


,ABN,Index_Link,Legal_Name,Town/Suburb,State,Status,Size,website,revenue,expenses,causes,lastreport,religious_ref,Number_of_Responsible_Persons,Registration_Date,Date_Organisation_Established
